In [76]:
import pandas as pd

Firstly, we need to load the data.

In [77]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')
oil = pd.read_csv('../data/oil.csv')
stores = pd.read_csv('../data/stores.csv')
transaction = pd.read_csv('../data/transactions.csv')
holidays = pd.read_csv('../data/holidays_events.csv')

Then we need to view the shape and the columns contained in each file / dataframe so we can better recognise how to merge the data, and what we can actually use.

In [78]:
print(f"Train data:\n {train_data.head()}")
print(f"Test data:\n {test_data.head()}")
print(f"Oil data:\n {oil.head()}")
print(f"Store data:\n {stores.head()}")
print(f"Transaction data:\n {transaction.head()}")
print(f"Holidays data:\n {holidays.head()}")

Train data:
    id        date  store_nbr      family  sales  onpromotion
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0
1   1  2013-01-01          1   BABY CARE    0.0            0
2   2  2013-01-01          1      BEAUTY    0.0            0
3   3  2013-01-01          1   BEVERAGES    0.0            0
4   4  2013-01-01          1       BOOKS    0.0            0
Test data:
         id        date  store_nbr      family  onpromotion
0  3000888  2017-08-16          1  AUTOMOTIVE            0
1  3000889  2017-08-16          1   BABY CARE            0
2  3000890  2017-08-16          1      BEAUTY            2
3  3000891  2017-08-16          1   BEVERAGES           20
4  3000892  2017-08-16          1       BOOKS            0
Oil data:
          date  dcoilwtico
0  2013-01-01         NaN
1  2013-01-02       93.14
2  2013-01-03       92.97
3  2013-01-04       93.12
4  2013-01-07       93.20
Store data:
    store_nbr           city                           state type  cluster
0

So, the distinctive data we can use, so that we can get the best possible prediction results, is if we use as an index the store and the date where it is possible. We now need to merge the date and make sure we have a consistent time frame, so we want our input data to have a 1 day difference, between 2 data rows, we have to make sure our data is ordered by date.

In [79]:
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])
oil['date'] = pd.to_datetime(oil['date'])
transaction['date'] = pd.to_datetime(transaction['date'])
holidays['date'] = pd.to_datetime(holidays['date'])

train_data = train_data.merge(
    oil, how='left', on='date'
)


train_data = train_data.merge(
    stores,
    how='left',
    on=['store_nbr']
)

test_data = test_data.merge(
    oil, how='left', on='date'
)

test_data = test_data.merge(
    stores,
    how='left',
    on=['store_nbr']
)

We did not merge all the available data yet. If we pay attention to the dataframes, we notice that the transaction data have no available data about the dates we want to predict the value of, if we just try to merge those tables, in the test data, we will just a get a column filled with nan / null values, the only thing this can do is throw off our predictions. The data we get from this dataset though is highly valuable, so we cannot just not use it. What we can actually do is work with lags and window frames. The logic for that is to actually match future dates with past values. For the lag data, we can agree on a consistent lag time frame, and just use the say average of the last 7 days as the prediction for the current date.

In [80]:
train_data = train_data.merge(transaction, how='left', on=['date', 'store_nbr'])
test_data = test_data.merge(transaction, how='left', on=['date', 'store_nbr'])

train_data = train_data.sort_values(['date'])
test_data  = test_data.sort_values(['date'])


full = pd.concat([train_data, test_data])
full = full.sort_values(['date'])

full['transaction_lag_7'] = full.groupby('store_nbr')['transactions'].shift(7)
full['transaction_roll_mean_7'] = (
    full.groupby('store_nbr')['transactions']
    .transform(lambda x: x.shift(1).rolling(7, min_periods=1).mean())
)

train_data = full.loc[train_data.index]
test_data  = full.loc[test_data.index]

test_data.drop(columns=['transactions'], inplace=True)
train_data.drop(columns=['transactions'], inplace=True)

print(train_data['transaction_lag_7'].isna().count())
print(train_data['transaction_roll_mean_7'].isna().count())
print(train_data.size)

print(test_data['transaction_lag_7'].isna().count())
print(test_data['transaction_roll_mean_7'].isna().count())
print(test_data.size)

print(f"New train data:\n {train_data.head()}")
print(f"New test data:\n {test_data.head()}")

3029400
3029400
39382200
57024
57024
741312
New train data:
            id       date  store_nbr        family  sales  onpromotion  \
0           0 2013-01-01          1    AUTOMOTIVE    0.0            0   
0     3000888 2017-08-16          1    AUTOMOTIVE    NaN            0   
1194     1194 2013-01-01         42   CELEBRATION    0.0            0   
1194  3002082 2017-08-16         42   CELEBRATION    NaN            0   
1193     1193 2013-01-01         42  BREAD/BAKERY    0.0            0   

      dcoilwtico    city      state type  cluster  transaction_lag_7  \
0            NaN   Quito  Pichincha    D       13                NaN   
0           46.8   Quito  Pichincha    D       13                NaN   
1194         NaN  Cuenca      Azuay    D        2                NaN   
1194        46.8  Cuenca      Azuay    D        2                NaN   
1193         NaN  Cuenca      Azuay    D        2                NaN   

      transaction_roll_mean_7  
0                         NaN  
0  